In [ ]:
import fiona

In [ ]:
fname = 'Route from 2021-02-02 08 27_20210202082755.gpx'

In [ ]:
fiona.listlayers(fname)

In [ ]:
layer = fiona.open(fname, layer='tracks')

In [ ]:
from shapely.geometry import shape

In [ ]:
geom = layer[0]

In [ ]:
data = {'type': 'MultiLineString',
        'coordinates': geom['geometry']['coordinates']}

shp = shape(data)
shp

In [ ]:
import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.io import shapereader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER


def make_map(projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(9, 13),
                           subplot_kw=dict(projection=projection))
    gl = ax.gridlines(draw_labels=True)
    gl.xlabels_top = gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
import cartopy.io.img_tiles as cimgt

request = cimgt.OSM()

extent = [0.09, 0.12,
          51.43, 51.37]

fig, ax = make_map(projection=request.crs)
ax.set_extent(extent)

img = ax.add_image(request, 14)
s = ax.add_geometries(shp, ccrs.PlateCarree(),
                      facecolor='none',
                      edgecolor='crimson',
                      linewidth=2)